In [1]:
import gc
import pandas as pd
import numpy as np
import os

from tqdm import tqdm_notebook

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify

%matplotlib inline

In [2]:
# load pkls
df_sales = loadpkl('../feats/sales.pkl')
df_calendar = loadpkl('../feats/calendar.pkl')
df_sell_prices = loadpkl('../feats/sell_prices.pkl')
df_sub = pd.read_csv('../input/sample_submission.csv')

In [4]:
df_sales.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,demand_shift_0,demand_shift_1,demand_shift_2,demand_shift_365,demand_mean_7,demand_mean_30,demand_mean_60,demand_mean_90,demand_mean_180,demand_mean_365,demand_std_7,demand_std_30,demand_std_60,demand_std_90,demand_std_180,demand_std_365,demand_skew_7,demand_skew_30,demand_skew_60,demand_skew_90,demand_skew_180,demand_skew_365,demand_kurt_7,demand_kurt_30,demand_kurt_60,demand_kurt_90,demand_kurt_180,demand_kurt_365,demand_max_7,demand_max_30,demand_max_60,demand_max_90,demand_max_180,demand_max_365,demand_min_7,demand_min_30,demand_min_60,demand_min_90,demand_min_180,demand_min_365
60034805,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.0,0.0,0.0,0.0,0.0,0.033333,0.100000,0.166667,0.505556,0.564384,0.000000e+00,1.825742e-01,0.399152,0.479934,0.954245,0.919473,NaN,5.477226,4.168783,2.945247,2.422061,2.080052,NaN,30.000000,17.082559,7.912159,7.218466,5.608411,0.0,1.0,2.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
60034806,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.0,0.0,0.0,0.0,0.0,0.033333,0.150000,0.100000,0.050000,0.024658,0.000000e+00,1.825742e-01,0.404425,0.336873,0.242773,0.172076,NaN,5.477226,2.771292,3.570995,5.288628,7.697129,NaN,30.000000,7.638707,13.353341,30.478895,65.655286,0.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
60034807,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.0,0.0,0.0,0.0,0.0,0.033333,0.466667,0.700000,0.727778,0.547945,1.178825e-07,1.825742e-01,0.929127,0.988132,0.944437,0.868260,NaN,5.477226,2.330157,1.353494,1.294449,1.750819,NaN,30.000000,5.593750,1.330417,1.300009,2.948203,0.0,1.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
60034808,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.0,0.0,0.0,0.0,0.0,0.133333,0.566667,0.677778,1.011111,1.082192,3.441276e-08,5.713465e-01,0.980603,0.992418,1.290225,1.515066,NaN,4.782377,1.871299,1.396046,1.730906,2.157170,NaN,23.773557,2.856141,1.090236,4.538466,8.288495,0.0,3.0,4.0,4.0,8.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
60034809,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.666667,1.033333,1.277778,1.534247,0.000000e+00,1.093258e-07,1.445820,1.617669,1.753068,1.922125,NaN,NaN,2.491999,1.492216,1.857794,1.731767,NaN,NaN,6.510493,1.491610,4.345049,4.138674,0.0,0.0,7.0,7.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_calendar.tail()

,date,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,seasonality
1964,2016-06-15,11620,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1,0.962513
1965,2016-06-16,11620,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0,0.967028
1966,2016-06-17,11620,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0,0.971257
1967,2016-06-18,11621,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0,0.975200
1968,2016-06-19,11621,2,6,2016,d_1969,13.0,0.0,3.0,0.0,0,0,0,0.978856


In [6]:
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [9]:
df_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,demand_shift_0,demand_shift_1,demand_shift_2,demand_shift_365,demand_mean_7,demand_mean_30,demand_mean_60,demand_mean_90,demand_mean_180,demand_mean_365,demand_std_7,demand_std_30,demand_std_60,demand_std_90,demand_std_180,demand_std_365,demand_skew_7,demand_skew_30,demand_skew_60,demand_skew_90,demand_skew_180,demand_skew_365,demand_kurt_7,demand_kurt_30,demand_kurt_60,demand_kurt_90,demand_kurt_180,demand_kurt_365,demand_max_7,demand_max_30,demand_max_60,demand_max_90,demand_max_180,demand_max_365,demand_min_7,demand_min_30,demand_min_60,demand_min_90,demand_min_180,demand_min_365
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,HOBBIES_1_016_CA_1_validation,HOBBIES_1_016,HOBBIES_1,HOBBIES,CA_1,CA,d_1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,HOBBIES_1_022_CA_1_validation,HOBBIES_1_022,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,HOBBIES_1_023_CA_1_validation,HOBBIES_1_023,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,HOBBIES_1_029_CA_1_validation,HOBBIES_1_029,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,HOBBIES_1_032_CA_1_validation,HOBBIES_1_032,HOBBIES_1,HOBBIES,CA_1,CA,d_1,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,HOBBIES_1_036_CA_1_validation,HOBBIES_1_036,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,HOBBIES_1_044_CA_1_validation,HOBBIES_1_044,HOBBIES_1,HOBBIES,CA_1,CA,d_1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
